# Test sending data to Firefly through Flask

*Working from customReader.ipynb*


In [1]:
import numpy as np
import h5py

import sys
import json
import requests
from IPython.display import IFrame

In [3]:
%load_ext autoreload
%autoreload 2

from firefly_api.reader import SimpleReader

sys.path.insert(0, '/Users/ageller/VISUALIZATIONS/Firefly')
from FireflyFlaskApp import startFireflyServer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Start the server

In [5]:
startFireflyServer()

 * Restarting with fsevents reloader


SystemExit: 1

## Show Firefly in an IFrame

In [ ]:
url = "http://localhost:5000"
IFrame(url, width=1000, height=500)

### Create some data, and get the Firefly dict

In [2]:
## Here I'll create a grid of points in the shape of a cube
my_coords = np.linspace(-10,10,20)
xs,ys,zs = np.meshgrid(my_coords,my_coords,my_coords)
xs,ys,zs = xs.flatten(),ys.flatten(),zs.flatten()
coords = np.array([xs,ys,zs]).T

with h5py.File("foo.hdf5",'w') as handle:
    group = handle.create_group('group1')
    group['Coordinates'] = coords
print("output to foo.hdf5")

np.savetxt('foo.csv',coords)
print("output to foo.csv")

output to foo.hdf5
output to foo.csv


## Send this data to the Flask app

In [8]:
## test out the csv
fdir = '/Users/ageller/VISUALIZATIONS/Firefly/static/data/foo_csv'
#simple = SimpleReader("foo.csv",write_jsons_to_disk=False,extension='.csv')
simple = SimpleReader(fdir,write_jsons_to_disk=False,extension='.csv')
simple.sendDataViaFlask()

Opening 1 files and 1 particle types...
JSONdir is None, defaulting to /Volumes/highnoon2go/highnoon/VISUALIZATIONS/Firefly/static/data/Data
outputting: 
ParticleGroup: foo
Contains 7999 particles (7999 after decimation) and 0 tracked fields
posting...data posted!


In [7]:
## test out the hdf5
fdir = '/Users/ageller/VISUALIZATIONS/Firefly/static/data/foo_hdf5'
#simple = SimpleReader("foo.hdf5",write_jsons_to_disk=False,extension='.hdf5')
simple = SimpleReader(fdir,write_jsons_to_disk=False,extension='.hdf5')
simple.sendDataViaFlask()

Opening 1 files and 1 particle types...
JSONdir is None, defaulting to /Volumes/highnoon2go/highnoon/VISUALIZATIONS/Firefly/static/data/Data
outputting: 
ParticleGroup: group1
Contains 8000 particles (8000 after decimation) and 0 tracked fields
posting...data posted!


In [5]:
## alternatively, make the request yourself (literally just copying the code from sendDataViaFlask())
port = 5000
print('sending to Firefly', sys.getsizeof(simple.JSON))
requests.post(f'http://localhost:{port:d}/data_input',json=simple.JSON)

sending to Firefly 322432


<Response [200]>

In [6]:
#test with FIRE data
fdir = '/Users/ageller/VISUALIZATIONS/FIREdata/snapdir_050/'
reader = SimpleReader(fdir, write_jsons_to_disk=False, decimation_factor=100)
reader.sendDataViaFlask()

Opening 1 files and 6 particle types...
JSONdir is None, defaulting to /Volumes/highnoon2go/highnoon/VISUALIZATIONS/Firefly/static/data/Data
outputting: 
ParticleGroup: PartType0
Contains 1066668 particles (10666 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType1
Contains 5860827 particles (58608 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType2
Contains 6175377 particles (61753 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType3
Contains 7732157 particles (77321 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType4
Contains 514118 particles (5141 after decimation) and 0 tracked fields
outputting: 
ParticleGroup: PartType5
Contains 1 particles (1 after decimation) and 0 tracked fields
posting...data posted!


*To kill the Firefly server.*

In [ ]:
# uncomment the next line
! ps aux | grep Firefly | awk '{print $2}' | xargs kill
